# Исследование надежности заемщиков


В ходе выполнения проекта необходимо провести предобработку данных, небольшой EDA, а также ответить на несколько вопросов, установленных заказчиком:

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Дополнительно приведём возможные причины возникновения пропусков в данных, а также причину выбора медианного значения для заполнения пропусков, вместо среднего.

**Описание данных:**

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Шаг 1. Загрузка данных и просмотр общей информации

Для начала, импортируем библиотеку `pandas`

In [1]:
import pandas as pd

Загрузим данные в переменную `data`

In [2]:
data = pd.read_csv('data.csv')

Посмотрим на данные. Выведем первые 5 строк методом `head()`

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Также ознакомимся с информацией о данных с помощью метода `info()`

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Шаг 2. Предобработка данных

### Удаление пропусков

Необходимо узнать количество пропусков в данных. Воспользуемся комбинацией методов `isna()` и `info()`

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах имеются пропущщенные значения. Один из них - `days_employed`. Пропуски в нём обработаем немного позже.  
Другой столбец с пропусками `total_income` - хранит данные о доходах клиентов. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можно медианным значением по каждому типу из столбца `income_type`.  

In [6]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == type), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`.  Избавимся от минуса с помощью метода `abs()`

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости ознакомимся с медианным значением трудового стажа `days_employed` в днях

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Оставим эти значения, как есть, так как этот столбец не понадобится нам для исследования.

Посмотрим на перечень уникальных значений столбца `children`

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Теперь значения в столбце корректные

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру

In [15]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

Дубликаты успешно удалены

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`

In [18]:
def categorize_income(income):

    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'

Применим функцию для создания нового столбца

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данные и ответ на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
#создаём сводную таблицу с группировкой по кол-ву детей
children_pivot = data.pivot_table(index='children', values='debt', aggfunc='mean')

#оформляем таблицу
children_pivot = children_pivot.rename(columns={'debt': '% должников'})
children_pivot.index.name = 'Количество детей'
children_pivot['% должников'] = (children_pivot['% должников'] * 100).round(2) 

#считаем процент должников среди родителей
parents_debt = data.loc[data['children'] != 0, 'debt'].mean()

display(children_pivot)
print('Средний процент должников, среди родителей хотя бы одного ребёнка: {:.2%}'.format(parents_debt))

,% должников
Количество детей,
0,7.54
1,9.23
2,9.45
3,8.18
4,9.76
5,0.00


Средний процент должников, среди родителей хотя бы одного ребёнка: 9.24%


**Вывод:** 

Судя по сводной таблице, в которой указан процент должников для каждого количества детей, значительной зависимости нет. Есть небольшая зависимость только от наличия детей: у родителей средний процент должников равен 9.24%, а у клиентов, которые ещё не имеют детей - 7.54%.

Разница в 2.3% кажется небольшой, но, это только один из нескольких показателей надёжности клиента. В сумме, все показатели дадут более существенную разницу. Проанализировав все параметры клиентов, можно будет более точно определить вероятность того, что кредит будет оплачен в срок.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
#создаём сводную таблицу с группировкой по семейному статусу
family_status_pivot = data.pivot_table(index='family_status', values='debt', aggfunc='mean')

#оформляем таблицу
family_status_pivot = family_status_pivot.rename(columns={'debt': '% должников'}) 
family_status_pivot.index.name = 'Семейный статус'
family_status_pivot['% должников'] = (family_status_pivot['% должников'] * 100).round(2)

#посчитаем средний процент должников среди клиентов, которые не были в официальном браке
not_married_avg = data.loc[(data['family_status'] == 'Не женат / не замужем') 
                           | (data['family_status'] == 'гражданский брак'), 'debt'].mean()

#посчитаем средний процент должников среди клиентов, которые были в официальном браке
married_avg = data.loc[(data['family_status'] == 'в разводе') 
                       | (data['family_status'] == 'вдовец / вдова') 
                       | (data['family_status'] == 'женат / замужем'), 'debt'].mean()

display(family_status_pivot)
print('Средний процент должников, среди клиентов, состоявших в официальном браке хотя бы раз: {:.2%}'.format(married_avg))
print('Средний процент должников, среди клиентов, не состоявших в официальном браке: {:.2%}'.format(not_married_avg))

,% должников
Семейный статус,
Не женат / не замужем,9.76
в разводе,7.06
вдовец / вдова,6.62
гражданский брак,9.31
женат / замужем,7.56


Средний процент должников, среди клиентов, состоявших в официальном браке хотя бы раз: 7.46%
Средний процент должников, среди клиентов, не состоявших в официальном браке: 9.49%


**Вывод:** 

Прослеживается зависимость от семейного статуса клиентов: те клиенты, которые хотя бы раз были в официальном браке, реже становятся должниками.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
#создаём сводную таблицу с группировкой по категориям дохода
income_pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=['mean', 'count'])    
    
#оформляем таблицу
income_pivot = income_pivot.rename(columns={'mean': '% должников', 'count': 'Кол-во человек', 'debt': ''}) 
income_pivot.index.name = 'Категория дохода'
income_pivot['% должников'] = (income_pivot['% должников'] * 100).round(2)
income_pivot = income_pivot.sort_values(by='% должников', ascending=False)
income_pivot

,% должников,Кол-во человек
,,
Категория дохода,,
E,9.09,22
C,8.50,15921
A,8.00,25
B,7.06,5014
D,6.02,349


Так как в категориях E и A очень мало человек для проведения анализа, можем объединить эти категории с D и B соответственно. Назовём категории "высокий доход" (200+ тысяч рублей дохода) и "низкий доход" (до 50 тысяч рублей дохода)

In [26]:
#создадим копию таблицы, чтобы не портить старую при изменении категорий
new_income_category_df = data 

#объединим и переименуем категории
new_income_category_df.loc[(new_income_category_df['total_income_category'] == 'A') 
         | (new_income_category_df['total_income_category'] == 'B'), 'total_income_category'] = 'высокий доход'
new_income_category_df.loc[(new_income_category_df['total_income_category'] == 'E') 
         | (new_income_category_df['total_income_category'] == 'D'), 'total_income_category'] = 'низкий доход'
new_income_category_df.loc[new_income_category_df['total_income_category'] == 'C', 'total_income_category'] = 'средний доход'

#создаём сводную таблицу с группировкой по новым категориям дохода
new_income_pivot = new_income_category_df.pivot_table(index='total_income_category', values='debt', aggfunc=['mean', 'count'])    
    
#оформляем таблицу
new_income_pivot = new_income_pivot.rename(columns={'mean': '% должников', 'count': 'Кол-во человек', 'debt': ''}) 
new_income_pivot.index.name = 'Категория дохода'
new_income_pivot['% должников'] = (new_income_pivot['% должников'] * 100).round(2)
new_income_pivot = new_income_pivot.sort_values(by='% должников', ascending=False)
display(new_income_pivot)


,% должников,Кол-во человек
,,
Категория дохода,,
средний доход,8.50,15921
высокий доход,7.06,5039
низкий доход,6.20,371


Построив сводную таблицу по уровням дохода, мы видим, что клиенты с высоким доходом реже допускают просрочки, чем клиенты со средним доходом. В таблице видно, что клиенты с низким доходом реже всего допускают просрочки, однако, в данной категории слишком мало людей, чтобы сделать по ней репрезентативный вывод. Доход точно влияет на вероятность оплаты кредита в срок, однако по текущим данным сложно сделать вывод о категории людей с низким доходом.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [27]:
#создаём сводную таблицу с группировкой по целям кредита (для подробного рассмотрения)
purpose_pivot = data.pivot_table(index='purpose', values='debt', aggfunc='mean')

#оформляем таблицу
purpose_pivot = purpose_pivot.rename(columns={'debt': '% должников'}) 
purpose_pivot.index.name = 'Цель кредита'
purpose_pivot['% должников'] = (purpose_pivot['% должников'] * 100).round(2)
purpose_pivot = purpose_pivot.sort_values(by='% должников', ascending=False)

display(purpose_pivot)

,% должников
Цель кредита,
получение дополнительного образования,11.49
сделка с автомобилем,10.99
получение высшего образования,10.82
сделка с подержанным автомобилем,10.60
профильное образование,10.19
свой автомобиль,9.94
заняться образованием,9.56
на покупку автомобиля,9.38
автомобили,9.24


In [28]:
#создаём сводную таблицу с группировкой по категориям кредита
purpose_category_pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc='mean')

#оформляем таблицу
purpose_category_pivot = purpose_category_pivot.rename(columns={'debt': '% должников'}) 
purpose_category_pivot.index.name = 'Категория кредита'
purpose_category_pivot['% должников'] = (purpose_category_pivot['% должников'] * 100).round(2)
purpose_category_pivot = purpose_category_pivot.sort_values(by='% должников', ascending=False)


display(purpose_category_pivot)

,% должников
Категория кредита,
операции с автомобилем,9.35
получение образования,9.25
проведение свадьбы,7.91
операции с недвижимостью,7.26


**Вывод:** 

На первой сводной таблице видно, что самыми безопасными для банка являются клиенты, которые берут кредит для покупки квартиры. А больше всего должников среди тех, кто берёт кредит на дополнительное образование.

На второй, обобщённой по категориям таблице, видно, что практически одинаково должников среди тех, кто берёт кредит на операции с автомобилем и на получение образования. Среди тех, кто берёт кредит на недвижимость, также меньше всего должников. 

#### 3.5 Возможные причины появления пропусков в исходных данных.

Пропуски были только в столбцах связанных с работой (количество дней трудоустройства и зарплата), я вижу только два варианта, откуда они могли возникнуть.

1. Многие люди просто не хотят указывать данные о своей работе. Однако, данный фактор один из самых важных для банка, при выдаче кредита. Поэтому, без этих данных, банк на вряд ли выдал бы кредит, скорее всего информация о работе обязательна при оформлении кредита.
2. Больше вероятен второй вариант: у кредитного отдела банка возникла техническая проблема при выгрузке данных о работе клиентов.

#### 3.6 Причина заполнения пропусков медианным значением

In [29]:
#создадим сводную таблицу с медианным, средним, максимальным и минимальным значением зарплаты
example = data.pivot_table(index='income_type', values='total_income', aggfunc=['min', 'max', 'mean', 'median'])
example = example.rename(columns={'total_income': ''})
display(example)

,min,max,mean,median
,,,,
income_type,,,,
безработный,59956,202722,131339.000000,131339
в декрете,53829,53829,53829.000000,53829
госслужащий,29200,910451,168728.454859,150447
компаньон,28702,2265604,199608.615811,172357
пенсионер,20667,735103,135320.753673,118514
предприниматель,499163,499163,499163.000000,499163
сотрудник,21367,1726276,159547.951975,142594
студент,98201,98201,98201.000000,98201


Как видно на таблице выше, значение зарплаты в одной категории может очень сильно отличаться, как, например, в категориях "сотрудник", "пенсионер", "госслужащий" и "компаньон". В этом наборе данных, среднее и медианное значение не очень сильно отличается. Однако, если бы, значений в каждой категории было меньше, аномально большие или маленькие значения гораздо сильнее влияли бы на среднее арифметическое. 

Следовательно, заполнять пропуски средним значением было бы некорректно, оно бы исказило данные. Поэтому, для заполнения пропущенных значений гораздо лучше подходит медиана. Она не исказит данные, даже при наличии аномально больших или маленьких значений.

### Шаг 4: общий вывод.

При помощи инструментов Python и библиотеки Pandas, были проанализированы данные о платёжеспособности клиентов и выявлены некоторые факторы, которые влияют на оплату кредита в срок:

- клиенты, которые были в официальном браке, реже допускают просрочки
- клиенты без детей реже допускают просрочки, чем клиенты с детьми, однако, количество детей не так важно
- на вероятность неоплаты кредита в срок влияет цель, на которую он был взят
- доход точно влияет на оплату кредита в срок, клиенты с высоким доходом реже допускают просрочки, чем клиенты со средним доходом, однако текущих данных недостаточно для репрезентативного вывода по категории людей с низким доходом.